In [ ]:
!pip install requests
!pip install --upgrade "ibm-watson>=6.1.0"

In [13]:
# Line 11: Replace with your IBM Watson API Key; Line 17: Replace url with your IBM URL

# IBM Watson
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
    import Features, EntitiesOptions

def getEntity(phrase):
  authenticator = IAMAuthenticator('{apikey}')
  natural_language_understanding = NaturalLanguageUnderstandingV1(
      version='2022-04-07',
      authenticator=authenticator
  )

  natural_language_understanding.set_service_url('{url}')

  response = natural_language_understanding.analyze(
      text=phrase,
      features=Features(entities=EntitiesOptions(limit=1))).get_result()

  return (json.dumps(response))

# Manually checking if an object is in user inputted phrase(can be replaced with advanced machine learning model using custom training data)
def planetsMentionedInPhrase(string): 
  if "sun" in (" " + string + " ") or "Sun" in (" " + string + " "):
    return "sun"
  if "earth" in (" " + string + " ") or "Earth" in (" " + string + " "):
    return "Earth"
  if "jupiter" in (" " + string + " ") or "Jupiter" in (" " + string + " "):
    return "Jupiter"
  if "saturn" in (" " + string + " ") or "Saturn" in (" " + string + " "):
    return "Saturn"
  if "mercury" in (" " + string + " ") or "Mercury" in (" " + string + " "):
    return "Mercury"
  if "mars" in (" " + string + " ") or "Mars" in (" " + string + " "):
    return "Mars"
  if "venus" in (" " + string + " ") or "Venus" in (" " + string + " "):
    return "Venus"
  if "uranus" in (" " + string + " ") or "Uranus" in (" " + string + " "):
    return "Uranus"
  if "neptune" in (" " + string + " ") or "Neutune" in (" " + string + " "):
    return "Neptune"
  if "pluto" in (" " + string + " ") or "Pluto" in (" " + string + " "):
    return "Pluto"
  if "nebula" in (" " + string + " ") or "Nebula" in (" " + string + " "):
    return "Nebula"
  return string
  

def entityInPhrase(phrase): # runs IBM watson, returns entity if there is one, returns original phrase if no found
  try:
    getEntity(phrase)
  except:
    return planetsMentionedInPhrase(phrase)
  ibmWatsonRes = json.loads(getEntity(phrase))     # Runs IBM watson to output entities and converts the string returned to a dictionary
  entityRes = ibmWatsonRes['entities']

  if len(entityRes) != 0:
    return (entityRes[0]['text'])
  else:
    return planetsMentionedInPhrase(phrase)

In [ ]:
# v.4 - Given phrase, uses IBM Watson to grab entity if applicable, and runs search and generates all images and respective description (on page 1)
import requests
import json
from IPython.display import Image

# gets user input and stores as search input to be searched in data base
userInput = input("Design a Prompt: ")
objectSearchingFor = entityInPhrase(userInput)  # Parses user input to only the object user wants image of
parameters = {
    "q": objectSearchingFor # sends the user desired object to the search parameter
}

# gets the data from nasa API
response = requests.get("https://images-api.nasa.gov/search?q={q}", params=parameters)

# Creates empty lists to store image link and respective description
imageLinksArray = []
imageDescription = []

# Filters through the list to hone in on only images and stores their links to array
for i in range(len(response.json()['collection']['items'])):
  dataType = response.json()['collection']['items'][i]['data'][0]['media_type']
  if dataType == "image":
    imageLink = response.json()['collection']['items'][i]['links'][0]['href']
    imageLinksArray.append(imageLink)

    imageDesc = response.json()['collection']['items'][i]['data'][0]['description']
    imageDescription.append(imageDesc)
  else:
    i+=1

# Prints out some basic info: the full links and number of links
print("Number of images found: " + str(len(imageLinksArray)))

# Verifies if there are any matches with user input
if len(imageLinksArray) != 0: # If there is at least one find
  # Displays all images in imageLinksArray and all respecitve descriptions in ImageDescription
  for i in range(len(imageLinksArray)):
    display(Image(imageLinksArray[i]))
    print(imageDescription[i])
    print()
else:
  print("No match :( Please try a different phrase")